# Laboratory 2.2.1

Welcome to Lab 2.2.1, which is a continuation of Lab 2.2 on Colab. In this lab, we will guide you on how to apply the model you just trained to detect objects in videos or via your laptop's camera.


## Instructions

Below is a detailed guide to help you understand how to apply YOLO to videos.


### Libraries

In [21]:
import numpy as np
import torch
import cv2
import math
import time
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Check GPU

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    print("GPU is available!")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("No GPU available, using CPU.")


### Loading the Detection Model

The `torch.hub.load` function in PyTorch is a powerful tool for loading models or modules from repositories shared on GitHub. It allows you to easily use pre-trained models or other modules without having to build them from scratch.

Structure of the `torch.hub.load` function:

```python
model = torch.hub.load(repo_or_dir, model, *args, **kwargs)
```

Parameters of the function:

1. **`repo_or_dir`**: 
   - This parameter specifies the GitHub repository containing the model you want to load. It is provided as a string, e.g., `'ultralytics/yolov5'`.
   - You can also specify a local path to the directory containing the model if the model has already been downloaded.

2. **`model`**: 
   - The name of the model or module you want to load from the repository. For example, `'custom'` for a custom model or `'yolov5s'` for a pre-trained YOLOv5s model.
   - Depending on the repository, this name may represent a specific model or a set of predefined models.

3. **`path`**: 
   - The path to the `.pt` file containing the trained model weights. This parameter is usually used when you want to load a pre-trained model that you have built or customized.

4. **`force_reload`**: 
   - This parameter is a boolean value (`True` or `False`).
   - When `force_reload=True`, the function will reload the model from the repository or the specified path even if the model has already been loaded before. This is useful when you want to avoid conflicts or ensure that you are using the latest version of the model.


**Exercise 1:** Complete the following code by filling in the [...]

In [ ]:
# force reload: avoid parameter conflicts when loading new models
model = torch.hub.load('ultralytics/yolov5', [...], path= [...], force_reload = [...])

### Testing with Images

Before working with videos, we will first apply the model to a few images to observe its effectiveness.

**`model(img_path)`**: This command uses the YOLOv5 model that was previously loaded to detect objects in the image.
- **`img_path`** is the path to the image you want to run object detection on.
- When you call the model with an image path like this, the model processes the image and returns the detection results as a `results` object.

**`test_detect.show()`**: This method displays the image with bounding boxes drawn around the detected objects.
- The detected objects are shown with information such as class labels and the confidence level of each object.


In [ ]:
img_path = [...]

In [ ]:
img = cv2.imread(img_path)

cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
test_detect = model(img_path)
test_detect.show()

### Work with Video

**Exercise 2:** Complete the exercise by filling in the [...]

In [ ]:
input_video_path = [...]  # Path to input video
output_video_path = [...]  # Path to output video

# Open input video
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video parameters
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create video writer to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frames from BGR to RGB
    rgb_frame = [...]

    # Predict with YOLOv5
    results = model([...])

    # Get the prediction result as a DataFrame
    df = results.pandas().xyxy[0]

    # Draw prediction boxes on the canvas
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = row['name']
        score = row['confidence']
        
        # Draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Draw class labels and confidence levels
        cv2.putText(frame, f'{label} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Record frames to output video
    out.write(frame)

# Free up resources
cap.release()
out.release()
cv2.destroyAllWindows()
